<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/meta_time_stack_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
End-to-end reasoning scaffold with:
- Analogy mapping
- Predicate encoding and rule firing
- Simple physics fall/impact check with rule override
- Temporal necessity check (□(Safe(M′)) → Adopt(M′))
- Memory SSM sketch, attention retrieval, conflict resolution
- Interrupt gate (HALT_AND_SAFE → CONTINUE)
- Reflective update: simulate_n_worlds + invariant preservation
- Time networks with cycle detection
- Datalog-styled rule example (fixed arity, no crash)

Notebook-safe: all logic is encapsulated in functions; top-level guarded by __main__.
Deterministic: seeded where randomness appears.
"""

from __future__ import annotations
from dataclasses import dataclass
from typing import List, Tuple, Dict, Optional, Set, Iterable
import math
import random
import sys


# -------------------------
# Utilities
# -------------------------

def print_section(title: str) -> None:
    bar = "=" * 3
    print(f"\n{bar} {title} {bar}")

def pretty_arrow(a: str, b: str) -> str:
    return f"{a} ↔ {b}"

def seeded(seed: int = 42):
    random.seed(seed)


# -------------------------
# 1) Analogy mapping
# -------------------------

def analogy_mapping(source: str, target: str, pairs: List[Tuple[str, str]]) -> None:
    print_section("Analogy")
    print(f"analogy = SME.match(source=\"{source}\", target=\"{target}\")")
    for a, b in pairs:
        print("→", pretty_arrow(a, b))


# -------------------------
# 2) Predicate encoding and rule firing
# -------------------------

@dataclass(frozen=True)
class Predicate:
    name: str
    args: Tuple[str, ...]

    def __str__(self) -> str:
        args_str = ",".join(self.args)
        return f"{self.name}({args_str})"

def parse_predicate(s: str) -> Predicate:
    # e.g., "LeftOf(ObjectA,ObjectB)"
    s = s.strip()
    name, rest = s.split("(", 1)
    args = rest.rstrip(")").split(",")
    args = tuple(arg.strip() for arg in args)
    return Predicate(name=name.strip(), args=args)

def evaluate_predicates(predicates: List[Predicate], knowledge: Dict[str, set]) -> bool:
    """
    Trivial rule firing: checks if all predicates appear in knowledge base.
    knowledge maps predicate name to set of arg-tuples.
    """
    return all(tuple(p.args) in knowledge.get(p.name, set()) for p in predicates)

def demo_predicates():
    print_section("Encode → Predicates → Rule firing")
    pred_strs = ["LeftOf(ObjectA,ObjectB)"]
    print(f"predicates = {pred_strs} # e.g., [\"Is(Red,ObjectA)\", \"LeftOf(ObjectA,ObjectB)\"]")
    preds = [parse_predicate(s) for s in pred_strs]
    kb = {
        "LeftOf": {("ObjectA", "ObjectB")},
        "Is": {("Red", "ObjectA")}
    }
    _ = evaluate_predicates(preds, kb)


# -------------------------
# 3) Falling → impact simulation
# -------------------------

@dataclass
class Cup:
    material: str  # e.g., "ceramic", "metal", "plastic"
    mass_kg: float

@dataclass
class FallContext:
    height_m: float
    surface: str  # e.g., "tile", "wood", "carpet", "grass"

def impact_energy_joules(mass_kg: float, height_m: float, g: float = 9.81) -> float:
    return mass_kg * g * height_m

def break_probability(cup: Cup, ctx: FallContext) -> float:
    """
    Heuristic model:
    - Base risk grows with impact energy.
    - Ceramic is brittle; surface hardness increases risk.
    """
    E = impact_energy_joules(cup.mass_kg, ctx.height_m)
    surface_factor = {
        "tile": 1.0,
        "concrete": 1.0,
        "wood": 0.8,
        "laminate": 0.8,
        "carpet": 0.35,
        "grass": 0.25,
    }.get(ctx.surface, 0.7)

    material_factor = {
        "ceramic": 1.0,
        "glass": 0.9,
        "porcelain": 0.85,
        "plastic": 0.2,
        "metal": 0.05,
    }.get(cup.material, 0.6)

    # Normalize energy into a logistic input
    # Reference: ~0.6 m ceramic on tile ~ notable break risk
    scale = 0.6 * 9.81 * 0.35  # tunable
    x = (E / max(scale, 1e-6)) * surface_factor * material_factor
    # Logistic to map to [0,1]
    p = 1.0 / (1.0 + math.exp(- (x - 1.0)))
    return max(0.0, min(1.0, p))

def likely_break_rule(cup: Cup, ctx: FallContext, threshold_h: float = 0.6) -> bool:
    return (cup.material == "ceramic") and (ctx.height_m >= threshold_h)

def demo_fall_scenario():
    print_section("Falling → impact simulation check")
    cup = Cup(material="ceramic", mass_kg=0.30)
    ctx = FallContext(height_m=0.8, surface="carpet")

    p_break = break_probability(cup, ctx)
    print(f"Impact energy (J): {impact_energy_joules(cup.mass_kg, ctx.height_m):.3f}")
    print(f"Break probability (heuristic): {p_break:.3f}")

    print("\n=== Physical rule: Likely break if ceramic cup falls from height ===")
    likely = likely_break_rule(cup, ctx)
    # Decision policy: if soft surface + mid risk, predict not broken despite "likely" rule
    decision = (p_break > 0.6) and (ctx.surface not in {"carpet", "grass"})
    final_outcome = bool(decision and likely)

    print(f"Cup break? {final_outcome}")


# -------------------------
# 4) Temporal necessity: □(Safe(M′)) → Adopt(M′)
# -------------------------

def always_safe(model_name: str, environments: Iterable[str]) -> bool:
    # Placeholder: all tested envs pass safety gates
    return True

def adopt_model(model_name: str, safe: bool) -> bool:
    return bool(safe)

def demo_temporal_necessity():
    print_section("Temporal necessity: □(Safe(M′)) → Adopt(M′)")
    model = "M′"
    envs = ["dev", "staging", "prod"]
    safe = always_safe(model, envs)
    adopted = adopt_model(model, safe)
    print(f"Adopt({model})")
    print(f"Always safe: {safe} Adopted: {adopted}")


# -------------------------
# 5) Memory SSM, attention retrieval, conflict resolution
# -------------------------

def demo_memory_ssm():
    print_section("Memory SSM, attention retrieval, conflict resolution")
    SSM = [
        [1.0,   0.485, 0.493],
        [0.485, 1.0,   0.482],
        [0.493, 0.482, 1.0]
    ]
    print(f"SSM[i,j] = {SSM}")

    # Simple symmetric attention window over 5 tokens (illustrative)
    retrieved_memory_vector = [0.322, 0.339, 0.339, 0.339, 0.322]
    print(f"retrieved_memory_vector = {retrieved_memory_vector}")

    # Conflict detection (toy): same scene labeled with two alternatives; pick A by tie-break
    scene_a = ("scene", 1, "A")
    scene_b = ("scene", 1, "B")
    # Force a conflict on identical scene ids
    conflict_detected = (scene_a[1] == scene_b[1])
    resolver = "A" if conflict_detected else "None"
    print(f"Conflict detected between scene:1 and scene:1 → resolve: {resolver}")


# -------------------------
# 6) Interrupt and reflective update gate
# -------------------------

class InterruptGate:
    def __init__(self):
        self.state = "HALT_AND_SAFE"

    def tick(self) -> str:
        out = self.state
        # After a HALT_AND_SAFE, we allow CONTINUE on next tick
        self.state = "CONTINUE"
        return out

def demo_interrupt_gate():
    print_section("Interrupt and reflective update gate")
    gate = InterruptGate()
    print(f"AGI_interrupt: {gate.tick()}")
    print(f"AGI_interrupt: {gate.tick()}")


# -------------------------
# 7) Reflective update: simulate_n_worlds + invariant preservation
# -------------------------

def simulate_invariants(n_worlds: int, success_rate: float) -> Tuple[int, float]:
    # Deterministic projection to avoid heavy sampling
    successes = int(round(n_worlds * success_rate))
    return successes, successes / n_worlds

def demo_reflective_update():
    print_section("Reflective update mode: simulate_n_worlds + invariant preservation")
    n_worlds = 10000
    target_rate = 0.9985  # 99.85%
    successes, frac = simulate_invariants(n_worlds, target_rate)
    print(f"Invariants preserved in {frac*100:.2f}% of worlds")
    # Deployment threshold (strict): require >= 99.9%
    deploy = frac >= 0.999
    print(f"Deploy: {deploy}")


# -------------------------
# 8) Time networks: looped causality skeleton
# -------------------------

def ancestors_with_cycle(start: str, edges: Dict[str, List[str]]) -> Tuple[List[str], bool]:
    visited: Set[str] = set()
    stack: Set[str] = set()
    order: List[str] = []
    has_cycle = False

    def dfs(u: str):
        nonlocal has_cycle
        if u in stack:
            has_cycle = True
            return
        if u in visited:
            return
        stack.add(u)
        visited.add(u)
        order.append(u)
        for v in edges.get(u, []):
            dfs(v)
        stack.remove(u)

    dfs(start)
    return order, has_cycle

def demo_time_networks():
    print_section("Time networks: looped causality skeleton")
    edges = {
        "E1_now": ["E2_future"],
        "E2_future": ["E3_future2"],
        "E3_future2": ["E2_future"],  # back-edge forms a cycle
    }
    nodes, has_cycle = ancestors_with_cycle("E3_future2", edges)
    # Present as in the sketch (include self, then discovered predecessors in DFS order)
    print(f"ancestors(E3_future2) = {nodes}")
    print(f"has_cycle: {has_cycle}")


# -------------------------
# 9) Datalog-styled rule example (fixed arity)
# -------------------------

# Unified 3-tuple fact representation: (relation, arg1, arg2_or_None)
Fact = Tuple[str, str, Optional[str]]

def future_event(x: str, facts: Set[Fact]) -> bool:
    """
    There exists y such that:
      causal_predecessor(y, x)
      and current(y)
      and not failed(y)
    """
    preds = {(a, b) for (rel, a, b) in facts if rel == "causal_predecessor" and b == x}
    rel_set = {(rel, a) for (rel, a, _) in facts}
    return any(
        ("current", y) in rel_set and ("failed", y) not in rel_set
        for (y, _) in preds
    )

def demo_datalog():
    print_section("Datalog-styled rule example")
    facts: Set[Fact] = {
        ("causal_predecessor", "Y1", "X1"),
        ("current", "Y1", None),
        # ("failed", "Y1", None),  # keep commented to satisfy the rule
    }
    print("future_event(X1) =", future_event("X1", facts))


# -------------------------
# Main
# -------------------------

def main(argv: Optional[List[str]] = None) -> int:
    seeded(7)

    # 1) Analogy
    analogy_mapping(
        source="Solar System",
        target="Atom",
        pairs=[("Planet", "Electron"), ("Sun", "Nucleus"), ("Orbit", "Shell")]
    )

    # 2) Predicates
    demo_predicates()

    # 3) Falling / impact
    demo_fall_scenario()

    # 4) Temporal necessity
    demo_temporal_necessity()

    # 5) Memory SSM + conflict resolution
    demo_memory_ssm()

    # 6) Interrupt gate
    demo_interrupt_gate()

    # 7) Reflective update
    demo_reflective_update()

    # 8) Time networks
    demo_time_networks()

    # 9) Datalog rule example (fixed)
    # This no longer raises ValueError due to tuple arity mismatch.
    demo_datalog()

    print("\n=== Closing note ===")
    print("Pipeline executed end-to-end without arity errors. Safe to extend with richer rules or datasets.")
    return 0


if __name__ == "__main__":
    raise SystemExit(main(sys.argv[1:]))